In [1]:
from dewloosh.geom import PolyData
from dewloosh.geom.tri import triangulate
from dewloosh.geom.space import StandardFrame, PointCloud
from dewloosh.geom.utils import centralize, center_of_points
from dewloosh.math.linalg import linspace
from dewloosh.geom.tri import triangulate
from dewloosh.geom import grid
from dewloosh.geom.tri.triutils import get_points_inside_triangles
from dewloosh.geom.topo import remap_topo
from sectionproperties.pre.library.steel_sections import i_section as ISection
from sectionproperties.pre.library.steel_sections import circular_hollow_section as CHS
import numpy as np

In [2]:
sections = {}
sections['front'] = ISection(d=203, b=133, t_f=7.8, t_w=5.8, r=8.9, n_r=16)
sections['back'] = CHS(d=200, t=10, n=64)
sections = {}
#sections['left'] = CHS(d=200, t=10, n=64)

In [3]:
Lx, Ly, Lz = 300, 300, 300
points_per_edge = 3
mesh_size = Lx / (points_per_edge-1)

In [4]:
points = []
nTotalPoints = 0  # node counter

# corners
corner_coords = [
    [-Lx/2, -Ly/2, -Lz/2],
    [Lx/2, -Ly/2, -Lz/2],
    [Lx/2, Ly/2, -Lz/2],
    [-Lx/2, Ly/2, -Lz/2],
    [-Lx/2, -Ly/2, Lz/2],
    [Lx/2, -Ly/2, Lz/2],
    [Lx/2, Ly/2, Lz/2],
    [-Lx/2, Ly/2, Lz/2]
]
corner_coords = np.array(corner_coords)
points.append(corner_coords)
nTotalPoints += len(corner_coords)

# populate edges
nodes_of_edges = [
    [0, 1], [1, 2], [2, 3], [3, 0],
    [4, 5], [5, 6], [6, 7], [7, 4],
    [0, 4], [1, 5], [2, 6], [3, 7]
    ]
edge_coords = []
N = points_per_edge + 2
for nodes in nodes_of_edges:
    p0 = corner_coords[nodes[0]]
    p1 = corner_coords[nodes[1]]
    edge_coords.append(linspace(p0, p1, N)[1:-1])
edge_coords = np.vstack(edge_coords)
points.append(edge_coords)
nTotalPoints += len(edge_coords)

# faces
corners_of_faces = {
    'front' : [1, 2, 6, 5], 
    'back' : [0, 3, 7, 4], 
    'left' : [2, 3, 7, 6],  
    'right' : [0, 1, 5, 4],
    'bottom' : [0, 1, 2, 3], 
    'top' : [4, 5, 6, 7],  
}
edges_of_faces = {
    'front' : [1, 5, 9, 10], 
    'back' : [3, 7, 8, 11], 
    'right' : [0, 9, 4, 8],  
    'left' : [2, 6, 10, 11],
    'bottom' : [0, 1, 2, 3], 
    'top' : [4, 5, 6, 7],  
}

# center of face
def cof(id) : return center_of_points(corner_coords[corners_of_faces[id]])

# face frames
frames = {}
frames['front'] = StandardFrame(dim=3, origo=cof('front'))
rot90z = 'Body', [0, 0, np.pi/2], 'XYZ'
frames['left'] = frames['front'].fork(*rot90z).move(cof('left') - cof('front'))
frames['back'] = frames['left'].fork(*rot90z).move(cof('back') - cof('left'))
frames['right'] = frames['back'].fork(*rot90z).move(cof('right') - cof('back'))
rot_front_top = 'Body', [0, -np.pi/2, 0], 'XYZ'
frames['top'] = frames['front'].fork(*rot_front_top).move(cof('top') - cof('front'))
rot180y = 'Body', [0, np.pi, 0], 'XYZ'
frames['bottom'] = frames['top'].fork(*rot180y).move(cof('bottom') - cof('top'))

In [5]:
def xy_to_yz(coords):
    _c = np.zeros((coords.shape[0], 3))
    _c[:, 1] = coords[:, 0]
    _c[:, 2] = coords[:, 1]
    return _c

def yz_to_xy(coords):
    _c = np.zeros((coords.shape[0], 3))
    _c[:, 0] = coords[:, 1]
    _c[:, 1] = coords[:, 2]
    return _c
    
coords_grid, topo_grid = grid(size=(Lx*0.99, Ly*0.99), shape=(N, N), eshape='Q4', centralize=True)
Grid = PolyData(coords=coords_grid, topo=topo_grid)
grid_centers = Grid.centers()[:, :2]

In [6]:
faces = {}
for face in ['left']:
    
    f_frame = frames[face]    
    # collect points on corners and edges and their global indices
    # these indices are used later to remap the default topology
    # resulting from per-face triangulations
    f_coords_base = []
    f_inds_base = []
    _corner_inds = []
    for corner in corners_of_faces[face]:
        f_coords_base.append(corner_coords[corner])
        _corner_inds.append(corner)
    f_inds_base.append(np.array(_corner_inds, dtype=int))
    for edge in edges_of_faces[face]:
        inds = np.arange(points_per_edge) + edge * points_per_edge
        f_coords_base.append(edge_coords[inds])
        f_inds_base.append(inds + 8)
        
    # transform the coords (base coordinates) so far to face frame    
    f_coords_base = PointCloud(np.vstack(f_coords_base)).show(f_frame)
    f_coords_base = yz_to_xy(f_coords_base)[:, :2]
    
    # global indices and number of corner and edge points
    f_inds_base = np.concatenate(f_inds_base)
    nBasePoints = len(f_inds_base)
    
    # build face    
    if face in sections:
        # 1) create the mesh of the section
        # 2) rule out points of the base grid that the section covers
        # 3) add corner and edge nodes and do a triangulation 
        f_section = sections[face]
        f_section.create_mesh(mesh_sizes=[mesh_size])
        f_coords = centralize(np.array(f_section.mesh['vertices']))
        f_topo = np.array(f_section.mesh['triangles'].tolist())[:, :3]
        f_inds = get_points_inside_triangles(f_coords, f_topo, grid_centers).astype(bool)        
        f_coords = np.vstack([f_coords_base, f_coords, grid_centers[~f_inds]])
        f_coords, f_topo, _ = triangulate(points=f_coords)
    else:
        f_coords = np.vstack([f_coords_base, grid_centers])
        f_coords, f_topo, _ = triangulate(points=f_coords)
                
    # faces share some points, hence they must be consistent
    # in node numbering --> remap topology to match indices
    # of corner and edge nodes
    f_inds = np.arange(len(f_coords))
    nNewPoints = len(f_coords) - nBasePoints
    f_inds[:nBasePoints] = f_inds_base
    f_inds[nBasePoints:] = np.arange(nNewPoints) + nTotalPoints
    nTotalPoints += nNewPoints
    f_topo = remap_topo(f_topo, f_inds)
    
    # transform to global and append new coords to total collection
    f_coords = PointCloud(xy_to_yz(f_coords[nBasePoints:]), frame=f_frame).show()
    points.append(f_coords)
    
    # add topology to face to total collection
    faces[face] = f_topo 

In [7]:
cube = PolyData(coords=np.vstack(points))
for face in faces:
    cube[face] = PolyData(topo=faces[face])

In [8]:
frames['left'].origo()

ArrayBase([  0., 150.,   0.])

In [9]:
cube.topology()[:5]

array([[15, 16, 59],
       [65, 41, 42],
       [64, 16,  3],
       [ 3, 41, 64],
       [64, 59, 16]])

In [10]:
cube['left'].topology()[:5]

array([[15, 16, 59],
       [65, 41, 42],
       [64, 16,  3],
       [ 3, 41, 64],
       [64, 59, 16]])

In [11]:
cube.plot()

In [12]:
cube.coords().shape

(69, 3)

In [13]:
cube.topology().shape

(64, 3)

In [14]:
cube.topology().min(), cube.topology().max()

(2, 68)